In [1]:
import pandas as pd
import torch
import json
import itertools

from sentence_transformers import SentenceTransformer, util

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# load data

In [2]:
data = pd.read_csv('data/data.csv')
data

,file,ori_text,text,head,升级建议,购买建议,散热分析,猪王的良心结语,游戏测试,处理器分析,...,配置,优缺点,优点,缺点,head_short,屏幕方面,接口方面,噪音方面,续航方面,suggestion_short
0,聊一款拥有“薯条标”的高性能轻薄本 (2023_5_24 20_52_23).html,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,#聊一款拥有“薯条标”的高性能轻薄本# #聊一款拥有“薯条标”的高性能轻薄本# #原创#猪...,#聊一款拥有“薯条标”的高性能轻薄本# #聊一款拥有“薯条标”的高性能轻薄本# #原创#猪...,【升级建议】#这台笔记本电脑拆机并不难，卸下D壳的螺丝后即可轻松取下后盖。#双通道16GB#...,【购买建议】#1，对机身质感要求较高#2，需要大容量的存储空间#3，对续航要求较低#星Boo...,【散热分析】#上图是惠普星Book Pro14的拆机实拍图，双风扇双热管的组合。#室温25℃...,【猪王的良心结语】#从性价比和实用性的角度考虑，我最推荐的肯定还是“i5-13500H”版本...,NaN,NaN,...,#i7-13700H 处理器#16GB LPDDR5#5200MHz 内存#1TB 固态硬盘...,#优点！#1，A/C/D面金属，机身外壳质感较好#2，高负载下风扇噪音较小#3，搭载1TB固...,#1，A/C/D面金属，机身外壳质感较好#2，高负载下风扇噪音较小#3，搭载1TB固态硬盘，...,#1，实测续航表现一般#2，#原装90W适配器比较重#3，高负载下，核心温度较高#,#聊一款拥有“薯条标”的高性能轻薄本# #聊一款拥有“薯条标”的高性能轻薄本# #原创#猪...,屏幕方面#，实测色域容积为126.0%DCI-P3，色域覆盖100.0%DCI-P3，在Di...,接口方面#，机身左侧依次为3.5mm音频接口、USB-A 5Gbps、Mirco-SD卡槽；...,噪音方面#，当环境噪音为36.9dB时，它的满载人位分贝值为42dB。,续航方面#，它的PCmark10续航测试成绩为7小时57分钟，中高负载日常仿真脚本的运行时长...,【购买建议】#1，对机身质感要求较高#2，需要大容量的存储空间#3，对续航要求较低#星Boo...
1,聊一款昨天刚发布的高端游戏本 (2023_5_24 20_51_00).html,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,#聊一款昨天刚发布的高端游戏本# #聊一款昨天刚发布的高端游戏本# #原创#猪王# 笔吧评...,#聊一款昨天刚发布的高端游戏本# #聊一款昨天刚发布的高端游戏本# #原创#猪王# 笔吧评...,【升级建议】#这台笔记本电脑拆机比较困难，底盖和去年一样采用了“地包天”结构，卸下底面所有螺...,【购买建议】#1，对游戏性能要求很高#2，对屏幕素质要求很高#3，拥有金镶玉裹的生活条件#联...,【散热分析】#上图是联想拯救者 Y9000P 至尊版的拆机实拍图，双风扇+大面积VC均热板的...,【猪王的良心结语】#其实在测评拯救者之前，我对Y9000P的性能释放没什么期待，因为它肯定是...,【游戏测试】#今天这台电脑采用了GeForce RTX 4080笔记本电脑GPU，支持最新的...,NaN,...,NaN,#优点！#1，CPU液金导热+GPU满功耗，性能释放好#配合DLSS 3，能维持高帧率游戏#...,#1，CPU液金导热+GPU满功耗，性能释放好#配合DLSS 3，能维持高帧率游戏#2，采用...,#1，新增[超能模式]下噪音较高#2，键盘面偏软#3，系统优化欠佳，有驻留后台#,#聊一款昨天刚发布的高端游戏本# #聊一款昨天刚发布的高端游戏本# #原创#猪王# 笔吧评...,屏幕方面#，这块2.5K电竞屏的表现很不错，色域容积达到了107.3%sRGB，色域覆盖99...,接口方面#，机身左侧依次为雷电4、USB-A 5Gbps；#机身右侧有USB-A 5Gbps...,噪音方面#，野兽模式下满载噪音为52dB，新增的超能模式满载噪音58dB，全速模式下进一步提...,NaN,【购买建议】#1，对游戏性能要求很高#2，对屏幕素质要求很高#3，拥有金镶玉裹的生活条件#联...
2,聊一款重量较轻的大屏全能本 (2023_5_24 20_50_44).html,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,#聊一款重量较轻的大屏全能本# #聊一款重量较轻的大屏全能本# #原创#猪王# 笔吧评测室...,#聊一款重量较轻的大屏全能本# #聊一款重量较轻的大屏全能本# #原创#猪王# 笔吧评测室...,【升级建议】#这台笔记本电脑拆机并不难，但所有螺丝都藏在脚垫和小圆片下面，需要全部取下才能卸...,【购买建议】#1，对续航要求较高#2，想要屏幕够大的便携笔电#3，拥有五陵豪气的消费气质#L...,【散热分析】#上图是LG Gram#17 独显版的拆机实拍图，双风扇双热管的组合。#室温25...,【猪王的良心结语】#根据我自己的喜好，其实我并不希望LG gram推出“独显版”，因为这让它...,NaN,NaN,...,#i7-1360P 处理器#RTX3050#4GB 独立显卡（40W）#32GB LPDDR...,#优点！#1，续航表现较好，搭载90Wh大电池#2，#在同配置笔记本里，重量较轻#3，内存硬...,#1，续航表现较好，搭载90Wh大电池#2，#在同配置笔记本里，重量较轻#3，内存硬盘容量较大#,#1，高负载下散热表现一般#2，#价格较贵#3，#机身缺少金属质感#,#聊一款重量较轻的大屏全能本# #聊一款重量较轻的大屏全能本# #原创#猪王# 笔吧评测室...,NaN,NaN,NaN,NaN,【购买建议】#1，对续航要求较高#2，想要屏幕够大的便携笔电#3，拥有五陵豪气的消费气质#L...
3,聊一款超凉快的RTX4060游戏本 (2023_5_24 20_51_33).html,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,#聊一款超凉快的RTX4060游戏本# #聊一款超凉快的RTX4060游戏本# #原创#猪...,#聊一款超凉快的RTX4060游戏本# #聊一款超凉快的RTX4060游戏本# #原创#猪...,【升级建议】#这台笔记本电脑拆机不难，卸下底面螺丝即可揭开后盖。#双通道16GB#DDR5 ...,【购买建议】#1，对散热能力要求较高#2，对外部接口要求较高#3，对噪音不是很敏感#宏碁 掠...,【散热分析】#上图是宏碁 掠夺者#·#擎Neo的拆机实拍图，五热管双风扇的组合。#室温25℃...,【猪王的良心结语】#宏碁 掠夺者·擎Neo目前共推出了四款机型：#i5-13500HX+#R...,NaN,NaN,...,#i7-13700HX 处理器#RTX4060#8GB 独立显卡（140W）#16GB DD...,#优点！#1，散热规格较高#2，#接口比较丰富，搭载双雷电4#3，屏幕素质不错，亮度高#缺点...,#1，散热规格较高#2，#接口比较丰富，搭载双雷电4#3，屏幕素质不错，亮度高#,#1，性能释放比较保守#2，满载噪音较大，风扇策略需要优化#3，电源适配器较重#,#聊一款超凉快的RTX4060游戏本# #聊一款超凉快的RTX4060游戏本# #原创#猪...,屏幕方面#，它采用了一块16:10比例的电竞屏，实测色域容积105.9%sRGB，色域覆盖9...,接口方面#，机身左侧依次为RJ45网口、USB-A 5Gbps、Micro-SD卡槽、3.5...,噪音方面#，它的满载人位分贝值为61.8dB，强冷模式下为62.1dB。,NaN,【购买建议】#1，对散热能力要求较高#2，对外部接口要求较高#3，对噪音不是很敏感#宏碁 掠...
4,聊一款性能强悍的AMD游戏本 (2023_5_24 20_49_48).html,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,#聊一款性能强悍的AMD游戏本# #聊一款性能强悍的AMD游戏本# #原创#猪王# 笔吧评...,#聊一款性能强悍的AMD游戏本# #聊一款性能强悍的AMD游戏本# #原创#猪王# 笔吧评...,【升级建议】#这台笔记本电脑拆机不难，卸下底面所有螺丝即可取下后盖，注意连接灯条的两根排线。...,【购买建议】#1，对游戏性能要求极高#2，对散热能力要求较高#3，真正的膏腴子弟#ROG 魔...,【散热分析】#上图是ROG 魔霸7 PLUS超能版的拆机实拍图，双风扇加大面积均热板的组合，...,【猪王的良心结语】#之前我们在ROG 冰刃7双屏上见到过锐龙9 7945HX处理器，这次依靠...,NaN,【CPU简析】#上面两张图是锐龙9 7945HX的跑分整理，这颗16核的处理器在超大规格的加...,...,#锐龙9 7945HX 处理器#RTX4090 16GB 独立显卡（175W）#16GB D...,#优点！#1，高负载下表面温度低#2，双液金导热，手动模式性能释放较强#3，同级产品中价格不...,#1，高负载下表面温度低#2，双液金导热，手动模式性能释放较强#3，同级产品中价格不离谱#,#1，高转速下风扇有轻微哨声#2，接口数量较少且右侧无

### basic info

In [3]:
base = pd.read_csv('./data/info.csv').fillna('')
base = base[['品牌', '型号', '外观']]

### extra-info

In [4]:
df = pd.read_csv('./data/info-extra.csv')
df.head(1)

,file,屏幕,性能,散热分析,升级难度,可支持接口,噪音,续航
0,聊一款拥有“薯条标”的高性能轻薄本 (2023_5_24 20_52_23).html,"{""结论"": ""惠普星Book Pro 14是一款轻薄本"",\n""亮度"": ""未描述"",\n...","{\n""结论"": ""高性能轻薄本性能表现一般"",\n""是否独立显卡"": ""未描述"",\n""办...","{\n ""结论"": ""惠普星Book Pro14的散热表现中规中矩，可以达到稳定高性能与低...","{\n ""结论"": ""升级难度较低"",\n ""原因分析"": ""拆机并不难，可以轻松取下后...","[\n ""3.5mm音频接口"",\n ""USB-A 5Gbps"",\n ""...","{\n ""结论"": ""较低"",\n ""满载人位分贝值"": 42\n}","{\n ""结论"": ""一般"",\n ""PCmark10成绩"": ""7小时57分钟""\n}"


In [5]:
extra = {}
for col in df.columns[1:]:
    temp = []
    for i, item in df[col].items():
        try:
            d = json.loads(item)
            temp.append(d)
        except Exception as e:
            print(i, col, item, e)
            temp.append({})
    extra[col] = temp

9 散热分析 抱歉，根据提供的评测文章未找到该产品的散热分析描述。 Expecting value: line 1 column 1 (char 0)
20 散热分析 未描述。 Expecting value: line 1 column 1 (char 0)
21 散热分析 未描述。 Expecting value: line 1 column 1 (char 0)
23 散热分析 未描述。 Expecting value: line 1 column 1 (char 0)
27 散热分析 未描述。 Expecting value: line 1 column 1 (char 0)
34 散热分析 未描述。 Expecting value: line 1 column 1 (char 0)
35 散热分析 未描述。 Expecting value: line 1 column 1 (char 0)
39 散热分析 未描述。 Expecting value: line 1 column 1 (char 0)
41 散热分析 未描述。 Expecting value: line 1 column 1 (char 0)
2 可支持接口 未描述 Expecting value: line 1 column 1 (char 0)
2 噪音 未描述 Expecting value: line 1 column 1 (char 0)


In [6]:
df_extra = df[['file']]

In [7]:
screen = pd.DataFrame(extra['屏幕']).rename(columns={'结论': '屏幕总结'}).iloc[:, :-1]
df_extra = df_extra.join(screen)

In [8]:
performance = pd.DataFrame(extra['性能']).rename(columns={'结论': '性能总结', '是否独立显卡': '独立显卡'}).iloc[:, :-1].replace({True: '是', False: '否'})
df_extra = df_extra.join(performance)

In [9]:
cooling = pd.DataFrame(extra['散热分析']).rename(columns={'结论': '散热分析'})['散热分析'].fillna('')
df_extra = df_extra.join(cooling)

In [10]:
expand = pd.DataFrame(extra['升级难度']).rename(columns={'结论': '升级难度'})['升级难度']
df_extra = df_extra.join(expand)

In [11]:
port = pd.Series([','.join(x) for x in extra['可支持接口']], name='可支持接口')
df_extra = df_extra.join(port)

In [12]:
sound = pd.DataFrame(extra['噪音']).rename(columns={'结论': '噪音'})['噪音'].fillna('')
df_extra = df_extra.join(sound)

In [13]:
battery = pd.DataFrame(extra['续航']).rename(columns={'结论': '续航'})['续航'].fillna('')
df_extra = df_extra.join(battery)

In [14]:
df_extra = df_extra.replace({'未描述': ''})

### concat

In [15]:
df = df_extra.join(base)
df

,file,屏幕总结,亮度,分辨率,色彩,性能总结,独立显卡,办公性能,游戏性能,散热分析,升级难度,可支持接口,噪音,续航,品牌,型号,外观
0,聊一款拥有“薯条标”的高性能轻薄本 (2023_5_24 20_52_23).html,惠普星Book Pro 14是一款轻薄本,,2.8K,,高性能轻薄本性能表现一般,,一般,,惠普星Book Pro14的散热表现中规中矩，可以达到稳定高性能与低噪音的平衡。,升级难度较低,"3.5mm音频接口,USB-A 5Gbps,Mirco-SD卡槽,HDMI2.0,USB-C...",较低,一般,惠普,星Book Pro 14,星Book Pro拥有三面金属机身和“薯条LOGO”
1,聊一款昨天刚发布的高端游戏本 (2023_5_24 20_51_00).html,联想拯救者Y9000P至尊版是一款高端游戏本，屏幕具有16.1英寸、2560×1600分辨率...,,2560×1600,,高端游戏本性能表现良好,是,良好,强大,联想拯救者 Y9000P 至尊版的散热表现非常出色，可以承受高功耗的性能释放，但可能带来一定...,升级难度高,"雷电4,USB-A 5Gbps,3.5mm音频接口,USB-C 10Gbps（支持DP输出和...",较高,,联想,拯救者 Y9000P 至尊版,
2,聊一款重量较轻的大屏全能本 (2023_5_24 20_50_44).html,LG gram 17独显版是一款重量较轻的大屏全能本,最大亮度490nit,,色域容积102.4%DCI-P3，色域覆盖96.6%DCI-P3，在DisplayP3模式下...,大屏全能本性能表现一般,是,一般,,LG gram 17 独显版的散热表现一般，键盘温度偏高。,中等难度,,,,LG,gram 17 独显版,文章提到LG gram 17独显版的机身缺少金属质感
3,聊一款超凉快的RTX4060游戏本 (2023_5_24 20_51_33).html,屏幕亮度高，色彩表现不错。,高,,不错,性能释放比较保守，满载噪音较大，散热规格高，但风扇策略需要优化,是,一般,,宏碁 掠夺者·擎Neo的散热表现不错，温度特别低，但风扇噪音又偏高。,较容易,"RJ45网口,USB-A 5Gbps,Micro-SD卡槽,3.5mm音频接口,防盗锁口,U...",较高,,宏碁,掠夺者·擎Neo,宏碁 掠夺者·擎Neo具有散热规格较高的特点，包括堆热管、鳍片、均热板，以及风扇转速拉满等散...
4,聊一款性能强悍的AMD游戏本 (2023_5_24 20_49_48).html,ROG魔霸7 Plus超能版是一款性能强悍的游戏本,,,,高负载下表面温度低，手动模式性能释放较强，价格较低,是,,对游戏性能要求极高,ROG 魔霸7 Plus超能版散热表现不错,较容易,"USB-A 5Gbps,3.5mm音频接口,USB-C 10Gbps（均支持DP视频输出，其...",一般,一般,"""ROG""（华硕的游戏品牌）",魔霸7 Plus超能版,文本中提到ROG 魔霸7 Plus超能版具有双液金导热、手动模式性能释放较强的特点。\n文本...
5,聊一款普及32GB大内存的商务本 (2023_5_24 20_52_30).html,ThinkBook 14+是一款商务本，屏幕素质较好,,,,标配32GB内存，接口多，屏幕素质较好,,,,ThinkBook 14+ 整体性能调校偏激进，散热表现一般,升级难度低,"USB-C（支持PD充电与视频输出）,USB-A 5Gbps,HDMI2.0,雷电4,3.5...",较低,一般,联想,ThinkBook 14+ 2023,"联想 ThinkBook 14+ 2023 左滑看接口 机身左侧 机身右侧""：这部分描述了该..."
6,聊一款有进步的RTX4060游戏本 (2023_5_24 20_52_15).html,惠普VICTUS光影精灵9是一款有进步的游戏本，但未描述屏幕具体性能。,,,,配置进步，CPU规格提升，屏幕刷新率提升，显卡性能完整发挥,是,,,光影精灵9整机散热表现良好，游戏常用的WASD附近温度相对不错。,升级难度低,"RJ45网线接口,USB-A 5Gbps,3.5mm音频接口,USB-C 5Gbps（支持1...",较高,,惠普,VICTUS光影精灵9,惠普VICTUS光影精灵9 左滑看接口 机身左侧 机身右侧
7,聊一款性能激进的精致笔记本 (2023_5_24 20_51_20).html,屏幕素质好，键盘和触控板手感不错。,,3K,,高性能，屏幕素质好，CPU性能释放较强,是,,,YOGA Pro 14s整机散热表现良好，日常使用开启均衡模式够用。,升级难度中等,"雷电4（支持PD充电和DP视频输出）,USB-C 10Gbps（支持PD充电和DP视频输出）...",一般,一般,联想,YOGA Pro 14s,联想 YOGA Pro 14s 左滑看接口 机身左侧 机身右侧这台YOGA Pro 14s ...
8,聊一台拥有12个大核心的游戏本 (2023_5_24 20_51_41).html,机械革命蛟龙16 Pro是一款搭载R9 7845HX处理器的游戏本，具备2K240Hz屏幕和...,,2K240Hz,,CPU性能较强，屏幕素质不错,是,,,机械革命 蛟龙16 Pro整机散热表现中规中矩，键盘大部分区域处于高温状态，且风扇噪音很大。,升级难度较高,"USB2.0接口,3.5mm音频接口,USB-A 10Gbps,RJ45（2.5GbE）,H...",较高,一般,机械革命,机械革命 蛟龙16 Pro,机械革命 蛟龙16 Pro 左滑看接口 机身左侧 机身右侧 机身后部
9,聊一款新发布的120Hz 2.8K OLED大屏全能本 (2023_5_24 20_52_4...,华硕无畏Pro15 锐龙版的最大特点是屏幕，采用了一款高分辨率、高刷新率的OLED屏，面板素...,支持全亮度DC调光,2880×1620分辨率的OLED屏,支持色域切换和HDR,屏幕素质高，性能释放激进,是,,,,升级难度低,"USB2.0接口,电源接口,USB3.2 Gen1 Type-A,HDMI2.1,全功能US...",一般,较好,华硕,华硕 无畏Pro15 锐龙版,


In [16]:
info = ['\n'.join([f'{k}: {v}' for k, v in row.items() if v]) for _, row in df.iterrows()]

In [17]:
infos = []
for i, row in df.iterrows():
    info = [f'{k}: {v}' for k, v in row.items() if v and k != 'file']
    for c in range(2, 5):
        for com in itertools.combinations(info, c):
            infos.append((i, '\n'.join(com), c))

In [18]:
df_infos = pd.DataFrame(infos, columns=['idx', 'describe', 'item_count'])
df_infos

,idx,describe,item_count
0,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n分辨率: 2.8K,2
1,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n性能总结: 高性能轻薄本性能表现一般,2
2,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n办公性能: 一般,2
3,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n散热分析: 惠普星Book Pro1...,2
4,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n升级难度: 升级难度较低,2
...,...,...,...
33148,41,噪音: 一般\n续航: 较好\n品牌: 苹果（Apple）\n型号: 苹果 MacBook ...,4
33149,41,"噪音: 一般\n续航: 较好\n品牌: 苹果（Apple）\n外观: ""苹果 MacBook...",4
33150,41,"噪音: 一般\n续航: 较好\n型号: 苹果 MacBook Pro 16\n外观: ""苹果...",4
33151,41,噪音: 一般\n品牌: 苹果（Apple）\n型号: 苹果 MacBook Pro 16\n...,4


In [151]:
# df_infos.to_csv('./data/info_comb.csv', index=False)

# laod model

In [19]:
embedder = SentenceTransformer('./data/simcse-model', device='cuda')
info_embeddings = embedder.encode(df_infos['describe'], convert_to_tensor=True)

In [23]:
query = "我想要一台外观特别、屏幕素质高、适合办公的电脑"
query_embedding = embedder.encode(query, convert_to_tensor=True)

In [24]:
cos_scores = util.cos_sim(query_embedding, info_embeddings)[0]
top_results = torch.topk(cos_scores, k=3)

In [25]:
for i, idx in enumerate(top_results.indices.tolist()):
    print(f'top {i + 1}')
    print(df_infos.at[idx, 'describe'])

top 1
办公性能: 良好
外观: 华硕ProArt 创16具有触控板左上方的一个旋钮，用户可以通过它结合软件执行各类快捷操作。
top 2
屏幕总结: 宏碁非凡X14是一款兼顾了性能与便携性的笔记本电脑，具有较高的屏幕规格。
办公性能: 相对较好，可满足一般办公和影音娱乐需求。
top 3
独立显卡: 是
办公性能: 相对较好，可满足一般办公和影音娱乐需求。


In [160]:
int(torch.argmax(cos_scores))

8375

In [154]:
df_infos.at[8375, 'describe']

'性能总结: 性价比很高的电脑，适合价格敏感型消费者。屏幕素质好，配色新颖。但噪音大，键盘温度较高，机身塑料感较强。\n独立显卡: 是'